In [146]:
#generation matrice adjacence a partit d'un fichier

def creer_matrice():
    try:
        file=open('mat_adjacence','r')
    except ValueError:
        print("Oops!  fichier non ouvert")
    #print(file.read)

    while 1:
        une_ligne = file.readline() #renvoie une ligne avec le \n à la fin
        if not une_ligne:
            break
        val=[int(x) for x in une_ligne.split()]
        mat.append(val)
    #print(mat)
    '''
    for l in range(len(mat)):
        for c in range(len(mat[l])):
            if(mat[l][c]==1):
                print(mat[l][c],"->",l," ",c)
    '''


In [147]:
#initialisation

#import random
#mat=[[0,2,1],[2,0,1],[1,2,0]]
#population=[]

def init():
    for i in range(0,5):
        liste_taille=list(range(0,len(mat[0])))
        random.shuffle(liste_taille)
        population.append([liste_taille,5])
    #print(population)
#init()

In [148]:
#mat=[[0,5,2],[4,0,3],[1,5,0]]

def fitness(individu):
    somme_indiv=0
    for i in range(len(individu)):
        v_depart=individu[i] 
        if(i+1==len(individu)):
            v_arrive=individu[0]
        else:
            v_arrive=individu[i+1]

        #print(v_depart,"->",v_arrive)

        for l in range(len(mat)):
            if (l==v_depart): 
                for c in range(len(mat[l])):
                    if(c==v_arrive):
                        somme_indiv=somme_indiv+mat[l][c]
                        #print(mat[l][c]," donne une dist de:",somme_indiv)
    return somme_indiv

#fitness([1,0,2])

In [149]:
def mort(pop):
    popula=[]
    for i in range(len(pop)):
        pop[i][1]=pop[i][1]-1
        if(pop[i][1]!=0):
            popula.append(pop[i])
    return popula

In [150]:
def selection(population):
    somme=0
    new_pop=[]
    for i in range(len(population)):
        somme=somme+fitness(population[i][0])
    moyenne=somme/len(population)
    for j in range(len(population)):
        if(fitness(population[j][0])<=moyenne):
            new_pop.append(population[j])
    return new_pop


In [151]:
import random

def croisement(pop):
    for i in range(0,6):
        #selection des parents
        indice_parent1=random.randint(0, len(pop)-1)
        indice_parent2=random.randint(0, len(pop)-1)

        parent1=pop[indice_parent1][0]
        parent2=pop[indice_parent2][0]

        enfant1=parent1.copy()
        enfant2=parent2.copy()

        nb_modif=random.randint(0, len(enfant1)-1)
        #print(nb_modif)
        indice_retire=[]
        for x in range(0,nb_modif):
            indice=random.randint(0,nb_modif)
            while(indice in indice_retire):
                indice=random.randint(0,nb_modif)
            indice_retire.append(indice)
        

        for x in range(len(indice_retire)):
            enfant1[indice_retire[x]]=99
            enfant2[indice_retire[x]]=99
            #print(pop[parent2][0],"et",enfant2)
        #print(enfant1,"   ",enfant2)
        
        for u in range(len(enfant1)):
            if(enfant1[u]==99):
                if(parent2[u] not in enfant1):
                    enfant1[u]=parent2[u]
                else:
                    for x in range(len(parent2)):
                        if parent2[x] not in enfant1:
                            enfant1[u]=parent2[x]

            if(enfant2[u]==99):
                if(parent1[u] not in enfant1):
                    enfant2[u]=parent1[u]
                else:
                    for x in range(len(parent1)):
                        if parent1[x] not in enfant2:
                            enfant2[u]=parent1[x]

        pop.append([enfant1,5])
        pop.append([enfant2,5])
        #print(enfant1,"   ",enfant2)
        #print(pop)

#pop=[[[0,1,2,3],5],[[1,2,3,0],5]]
#croisement(pop)
#print(pop)

In [152]:
#import random

def croisement_v2(population):
    for i in range(0,6):
        #selection des parents
        indice_parent1=random.randint(0, len(population)-1)
        indice_parent2=random.randint(0, len(population)-1)
        #print("indice du parent1:",indice_parent1,"  indice du parent 2:",indice_parent2)
        #print("taille de la pop:",len(population)-1)

        parent1=population[indice_parent1][0]
        parent2=population[indice_parent2][0]
        indice_same=random.randint(1, len(parent1)-1)
        enfant1=[]
        enfant2=[]
        enfant1_temp=[]
        enfant2_temp=[]

        for x in range(0,indice_same):
            enfant1_temp.append(parent1[x])
            enfant2_temp.append(parent2[x])

        for y in range(indice_same,len(parent1)):
            enfant1_temp.append(parent2[y])
            enfant2_temp.append(parent1[y])

        for a in range(0,len(enfant1_temp)):
            if enfant1_temp[a] not in enfant1:
                enfant1.append(enfant1_temp[a])
            if enfant2_temp[a] not in enfant2:
                enfant2.append(enfant2_temp[a])
        
        for b in range(0,len(parent1)):
            if b not in enfant1:
                enfant1.append(b)
            if b not in enfant2:
                enfant2.append(b)
        
        population.append([enfant1,5])
        population.append([enfant2,5])
        #print(population)

#pop=[[[0,1,2,3],5],[[1,2,3,0],5]]
#croisement_v2(pop)
#print(pop)

In [153]:
#import random

def mutation(pop):
    mut=[]
    while(len(mut)!=3):
        pos=random.randint(0, len(pop)-1)
        mut.append(pos)
    for i in mut:
        perm1=random.randint(0, len(pop[i][0])-1)
        perm2=random.randint(0, len(pop[i][0])-1)
        #print(pop[i],"->",perm1,"   ",perm2)
        val1=pop[i][0][perm1]
        val2=pop[i][0][perm2]
        temp=val2
        pop[i][0][perm2]=val1
        pop[i][0][perm1]=temp
        #temp=pop[i][0][perm2]
        #perm2=pop[i][0][perm1]
        #perm1=temp
#pop=[[[0,1,2,3],5],[[4,5,6,7],5],[[8,9,1,2],5],[[3,4,5,6],5]]
#mutation(pop)
#print(pop)

In [154]:
def verif(pop,valeur,chemin):
    for i in range(len(pop)):
        if(fitness(pop[i][0])<valeur):
            valeur=fitness(pop[i][0])
            chemin=pop[i][0]
    return (valeur,chemin)
#mat=[[0,3,4,2],[5,0,6,1],[4,6,0,3],[2,1,3,0]]
#pop=[[[0,1,2,3],5],[[3,1,2,0],5],[[0,3,1,2],5],[[3,2,1,0],5]]
#val,chemin=verif(pop,999)
#print(val,"  ",chemin)

In [155]:
def algo_genetique(population):
    val_min=9999
    chemin_min=[]
    time=0
    creer_matrice()
    init()
    population=selection(population)
    val_min,chemin_min=verif(population,val_min,chemin_min)

    while(time<50 and len(population)>=2):
        time+=1
        croisement_v2(population)
        mutation(population)
        population=selection(population)
        population=mort(population)
        val_min,chemin_min=verif(population,val_min,chemin_min)
    print("Chemin min: ",chemin_min," pour un cout de: ",val_min)
    print("Fin")

In [156]:
import random
mat=[]
population=[]

algo_genetique(population)

Chemin min:  [2, 0, 1, 3]  pour un cout de:  8
Fin
